<h1> How I learned to stop worrying and love uncertainty </h1>

An introductory workshop on using data and simulation to make decisions when you don't know everything.

<b>author</b>: Parag Rastogi; Minu Agarwal <b>venue</b>: CEPT University, Ahmedabad; <b>date</b>: 8th-9th January, 2024.

Run each module one-by-one by either using the <kbd>run cell</kbd> button above or pressing <kbd>Ctrl + Enter</kbd> when a cell is selected. Modules like this one are `Markdown` or `HTML` modules. `Markdown` which is a kind of text-encoding language while `HTML` is used to build web pages. These will not produce an output - instead you will see formatted text in the cell when you run one.


<h2>Problem Statement</h2>

Let us consider a real estate transaction: someone is buying an existing building or set of buildings from someone else. Call the buildings being purchased an "asset". So, we have at least two stakeholders: a buyer and a seller. It is quite common for the buyer and sellers of buildings, i.e., past and new owners, to the lease or rent the building out to occupiers. The occupiers may be companies for offices, residents for flats, institutions, retail/shops, etc. Finally, there may be banks or financial institutions who finance the transaction for the new owner, e.g., a home loan or commercial loan against property (mortgage). These are the stakeholders who own, use, and operate a building. Some specialists might be part of the picture as well, such as facilities managers, utility companies (water, electricity, gas, internet, etc.), security providers, and others. Finally, there is you: the architect or engineer. What role do you play?

Real estate transactions do not, conventionally, consider the energy or environmental performance of a building or asset. This creates an "assymetry" of information between the seller and the buyer. This asymettry could have some undesirable outcomes, such as: 
<ul>
<li>the new owner is unable to plan their energy expenditure</li>
<li>the asset may already violate local energy or pollution laws, or be in danger of doing so in the near future</li>
<li>the asset may be in danger of getting "stranded", i.e., its energy and emissions will exceed the limits set by a reasonable, time-defined path to net zero, and there will be limited to no options to sufficiently reduce them.</li>
</ul>



<h2>Initialisation</h2>

<ol>
<li>If you haven't read the wiki, I suggest starting there: <a href='https://github.com/paragrastogi/CEPT_Workshop_Mar2024/wiki'>Wiki</a>. </li>


<li> The data used in this workshop is accessible from inside your Google colab notebook by adding it to your Google Drive: <a href='https://drive.google.com/drive/folders/15iu53fxrFhbfc3AAyYkZMYkdKChd-TtT?usp=sharing'>Workshop Data Folder</a>. If working on your computer, download the data to your computer and put the correct paths in the scripts below. </li>

** UPDATE **
You will see three folders in there: <code>India_Ahmedabad</code>, <code>Switzerland_Geneva</code>, and <code>India_Dehradun</code>. Download Geneva and Dehradun for this example,  placing them in the top-level directory where this script and others are located, i.e., at the same level as <code>lib</code>. For example, on my computer this file is in <code>C:\GitWorkspace\CEPT_Workshop_Mar2024</code>.
** UPDATE **

</ol>

Everything should work ok on Mac or Windows computers. If you're the kind of person who works on Linux, you're probably fine to troubleshoot any issues on your own. 

In this exercise we will use a regression model (black-box model) to estimate the response of a building. We will do an exercise related to decision-making with uncertain data using the regression model as a proxy for physics-based modelling. The rest of the workshop is structured as follows:

<ol>
    <li> Load python modules and weather files. Plot key parameters.
    <li> Compose your portfolio and examine their data.
    <li> Aleatory uncertainty with black-box models.
</ol>

In [1]:
# Import various standard modules.
import glob, os, copy, datetime, pickle

# Computational modules.
import pandas as pd
import numpy as np

# Plotting modules.
import matplotlib.pyplot as plt
import seaborn as sns

# Get inline graphs .
%matplotlib inline

# Only useful for debugging, when you 
# need to reload external modules.
from importlib import reload

# Enable xkcd mode if you're a geek like Parag.
# plt.xkcd()

# Import a custom read-write function for weather files.
import lib.wfileio as wf

# import a file of small helpers I've written. Call it helper.
import lib.petites as helper

# Import an awesome colour palette. Call it colours.
import lib.default_colours as colours

# Set the random seed to a specific value so
# the experiment is repeatable.
# See https://en.wikipedia.org/wiki/Random_seed
# for more information on what this means.
np.random.seed(42)

In [3]:
# Read weather data from location.

# I've used Ahmedabad as an example here - feel free to download weather data for any other city  
# from http://climate.onebuilding.org/default.html if you like.

pathWthrFolder = './data/ahmedabad' 
list_wfiles = glob.glob(pathWthrFolder+'/*.epw')

# Python can interpret the Unix file separator, the 'forward-slash' (/), on all platforms. 
# That is, if you consistently use '/', the paths are automatically constructed based on the OS.
# If you want to use the Windows back-slash, make sure to precede the path string with an 'r'.

# The small program get_weather stores data from the incoming weather file as a dataframe. 
# It outputs three things but I'm only using the first output for now, so I've put in underscores
# to indicate that the second and third outputs should not be assigned to a variable in memory.

# Declare a list.
listDfWthr = list()

for file in list_wfiles:
    
    gen_temp, _, _ = wf.get_weather('amd', file)
    listDfWthr.append(gen_temp)
    
# Create a dataframe from list but also keep the list - useful for plotting later. 
dfW = pd.concat(listDfWthr)

/Users/prastogi/Gitworkspace/CEPT_Workshop_Mar2024/lib/wfileio.py:350: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(type(wdata[col][0]), str):
/Users/prastogi/Gitworkspace/CEPT_Workshop_Mar2024/lib/wfileio.py:350: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(type(wdata[col][0]), str):
/Users/prastogi/Gitworkspace/CEPT_Workshop_Mar2024/lib/wfileio.py:350: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if 

In [ ]:


weather1 = weather.drop(["timestamp","site_id","cloudCoverage","windDirection"], axis=1)



print('DEHRADUN')

print(df_ddn.head())

print('======\r\n')

print('GENEVA')

print(df_gen.head())

print('======')

<h2> Black Box Models </h2>

A black-box model is any model (mathematical relationship) that can only be viewed in terms of its inputs and outputs. That is, we cannot "see" the inner workings or relationships that result in certain inputs giving certain outputs. It can be used to represent a physical system if sufficient data is available to characterise the relationship between the inputs and outputs of that system. In building simulation, for example, a black-box model could relate outside temperature to heating demand without any indication of how that demand is generated physically, i.e., without solving the equations of heat transfer.  

For this exercise we will use following equation for energy consumption in a given month ($E_m$):

$ E_m = \beta_{0, m} + \beta_{1, m} * \text{HDD}_{m} + \beta_{2, m} * \text{CDD}_{m}$  (1)

where HDD$_m$ is heating degree days in a given month (m), CDD$_m$ is cooling degree days in that same month, $\beta_{1,m}$ and $\beta_{2,m}$ represent the coefficients of each, and $\beta_{0,m}$ represents a constant load or consumption. This equation, thus, relates outdoor environmental conditions to energy usage. The coefficients of each weather/climate term represent the "effect" that that term has on energy usage. The constant (last coefficient) can be thought of as a base load in the context of most buildings, i.e., the part of the load not due to outdoor conditions.

In [ ]:
# Calculate the monthly mean values of weather parameters for Dehradun.

# These are the weather variables that will be used in the regression.
input_vars = ['tdb']

# Each weather parameter is grouped by month, and the monthly mean calculated.
# We only keep the relevant variables (relevant for the energy calculation below, that is.)
ddn_monthly = df_ddn.groupby(by=['month']).mean()[input_vars]
# Assign an index to the dataframe - useful for plotting.    
ddn_monthly.index = pd.unique(df_ddn['month'])

# Do the same for Geneva.
gen_monthly = list()

for yearlong in list_df_gen:
    
    temp_monthly = yearlong.groupby(by=['month']).mean()[input_vars]
    # Assign an index to the dataframe - useful for plotting.    
    temp_monthly.index = pd.unique(yearlong['month'])
    gen_monthly.append(temp_monthly)


In [ ]:
# Define a function to calculate monthly energy use using the equation given in Clarke (2001).
# This function takes as input weather data and (optionally) coefficients representing the 
# energy consumption correlated to that weather variable.

def loads_linear(xdata, *beta):
    
    cdd = xdata[0]
    hdd = xdata[1]
    
    if isinstance(beta[0], np.ndarray) or isinstance(beta[0], list):
        beta = beta[0]
                
    e = beta[0] + beta[1]*cdd + beta[2]*hdd
    
    return e

def loads_quad(xdata, *beta):
    
    cdd = xdata[0]
    hdd = xdata[1]
    
    if isinstance(beta[0], np.ndarray) or isinstance(beta[0], list):
        beta = beta[0]
            
    e = beta[0] + beta[1]*cdd + beta[2]*hdd + beta[3]*cdd**2 + beta[4]*hdd**2
    
    return e

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(16, 12)) 
ax = axes.flatten()
fig.tight_layout(pad=3, w_pad=2, h_pad=5)

# Find the index and value of the max and min temperatures.
ymax = list()
ymin = list()
xmax = list()
xmin = list()

ddn_temp_vals = df_ddn['tdb'].values
ymax.append(np.max(ddn_temp_vals))
xmax.append(np.argmax(ddn_temp_vals))
ymin.append(np.min(ddn_temp_vals))
xmin.append(np.argmin(ddn_temp_vals))

gen_temp_vals = df_gen['tdb'].values
ymax.append(np.max(gen_temp_vals))
ymin.append(np.min(gen_temp_vals))

plot_x = np.arange(0, df_ddn.shape[0])
# ax[0].plot(plot_x, ddn_smooth['tdb'].values, zorder=10, color=colours.blue)
ax[0].plot(plot_x, df_ddn['tdb'].values, zorder = 1, color=colours.orange, alpha=0.5)
ax[0].plot(plot_x, np.repeat(ddn_monthly['tdb'], 8760/12), color=colours.blackest, zorder=11)


for raw, monthly in zip(list_df_gen, gen_monthly):
    
    # ax[1].plot(plot_x, smooth['tdb'], zorder=10, color=colours.blue, alpha=0.75)
    
    raw_temp_vals = raw['tdb'].values
    
    if np.unique(raw.index.year)==2223:
        ax[1].plot(plot_x, raw_temp_vals, zorder = 12, color=colours.grey)
    else:
        ax[1].plot(plot_x, raw_temp_vals, zorder = 1, color=colours.orange, alpha=0.25)
    
    if ymin[1] in raw_temp_vals:
        xmin.append(np.argmin(raw_temp_vals))
    
    if ymax[1] in raw_temp_vals:
        xmax.append(np.argmax(raw_temp_vals))

for ax_temp in ax:
    ax_temp.set_ylabel('Dry bulb temperature [$^o$C]')
    ax_temp.set_xlim(plot_x[0], plot_x[-1])

ax[0].legend(['Hourly', 'Monthly'])
ax[1].legend(['Hourly'])
ax[1].set_xlabel('Hour of the year')
ax[0].set_title('Hourly values of temperatures for Dehradun')
ax[1].set_title('Hourly values of temperatures for Geneva')

ax[0].annotate('Hottest temperature \n in the TMY file\n',
    xy=(xmax[0], ymax[0]), arrowprops=dict(arrowstyle='->'), xytext=(4000, 10))
ax[0].annotate('Coldest temperature \n in the TMY file\n',
    xy=(xmin[0], ymin[0]), arrowprops=dict(arrowstyle='->'), xytext=(4000, 5))

ax[1].annotate('Hottest temperature \n in 36 years\n',
    xy=(xmax[1], ymax[1]), arrowprops=dict(arrowstyle='->'), xytext=(5000, -10))
ax[1].annotate('Coldest temperature \n in 36 years\n',
    xy=(xmin[1], ymin[1]), arrowprops=dict(arrowstyle='->'), xytext=(2000, -20))

plt.show()


<h2>Random coefficients, fixed inputs</h2>

For the first exercise, we fix the inputs to typical values and use random coefficients to specify the relationship between inputs and output. The simulators you use will almost never be random - for buildings at least - so this exercise represents a lack of knowledge about the coefficients. That is to say, <i>if we don't know the relationships, then they appear random</i>.

Every time you run the next cell, it should give you a different set of random coefficients. This in turn means that the values of 'energy consumption' will be different for <i>exactly the same inputs</i>. Clearly, this is not possible for a real building running under the exact same conditions. If you knew the 'answer', i.e., the true energy consumption, you could run this next cell until the coefficients gave you a perfect match of predicted energy use versus measured or true energy use. That is a dumb version of regression modelling, and will show you that pure guesswork, or 'brute force' is often a wasteful means of arriving at an answer. Clever algorithms have been developed that do this faster and with less calculations.

In [ ]:
# Select commonly-used balance points for HDD and CDD calculations.
bp= [15, 18] # degrees C

# Let us define a tolerance parameter to remove noise in the calculations.
# This is a common practice - to define a small number below which a value is considered zero.
# If a calculated Cooling or Heating Degree Day monthly sum is less than this number, we will set it to zero.
tol = 3

# You can also put a better filter by setting a cooling and heating 'season'. This can be done by,
# for example, setting CDD and HDD to zero in the respective months.

# Call random coefficients from the generator. Every time you run this, it will give you
# different numbers. Until you restart the kernel, in which case it will give you the same
# series of random numbers as the last time and every time before that.
random_coeffs = np.random.rand(3)

print(random_coeffs)

<h3>Random coefficients, Dehradun</h3>

Let's input random coefficients for HDD and CDD in the equation for energy usage in Dehradun. My intuition says to set the coefficient for HDD to zero when calculating cooling load, and vice-versa. <b>Is that correct?</b>

In [ ]:
cdd_temp_list = list()
hdd_temp_list = list()
cool_temp_list = list()
heat_temp_list = list()

for m in df_ddn.index.month.unique():
    cdd_temp,hdd_temp = dd(df_ddn.loc[df_ddn.index.month==m,'tdb'], bp, freq='H')
    
    if cdd_temp<=tol:
        cdd_temp = 0
    if hdd_temp<=tol:
        hdd_temp = 0
    
    cdd_temp_list.append(cdd_temp)
    hdd_temp_list.append(hdd_temp)
    
    # Calculate the monthly energy consumption with random coefficients.
    # My intuition says to set the coefficient for HDD to zero when calculating cooling load, and vice-versa.
    # Is that correct?
    cool_temp_list.append(loads_linear([cdd_temp, hdd_temp], [random_coeffs[0], random_coeffs[1], 0]))
    heat_temp_list.append(loads_linear([cdd_temp, hdd_temp], [random_coeffs[0], 0, random_coeffs[2]]))        
    
df_ddn_monthly = pd.DataFrame(
    dict(cdd=cdd_temp_list, hdd=hdd_temp_list, heat_random=heat_temp_list, cool_random=cool_temp_list),
    index=pd.date_range(start='2223-01-01 00:00', end='2223-12-31 23:00', freq='1M'))

Plot the outputs - does the load profile look reasonable? What does the plot of load versus Cooling or Heating Degree Day look like?

In [ ]:
fix, axes = plt.subplots(nrows=2, ncols=3, sharex=False, sharey=True, figsize=(16, 12)) 
ax = axes.flatten()
fig.tight_layout(pad=3, w_pad=2, h_pad=5)

fig.suptitle('Random coefficients')

e1 = ax[0].bar(df_ddn_monthly.index.month, df_ddn_monthly.loc[:, 'heat_random'], color=colours.blackest)

h1 = ax[1].plot(df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'hdd']>0, 'hdd'],
                df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'hdd']>0, 'heat_random'],
                marker='o', linewidth = 0, color=colours.red)
c1 = ax[2].plot(df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'cdd']>0, 'cdd'],
                df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'cdd']>0, 'heat_random'],
                marker='o', linewidth = 0, color=colours.blue)

ax[0].set_ylabel('Heating loads, Dehradun [$kWh/m^2$]')

ax[1].set_xlabel('HDD')
ax[2].set_xlabel('CDD')

e1 = ax[3].bar(df_ddn_monthly.index.month, df_ddn_monthly.loc[:, 'cool_random'], color=colours.blackest)

h1 = ax[4].plot(df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'hdd']>0, 'hdd'],
                df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'hdd']>0, 'cool_random'],
                marker='o', linewidth = 0, color=colours.red)
c1 = ax[5].plot(df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'cdd']>0, 'cdd'],
                df_ddn_monthly.loc[df_ddn_monthly.loc[:, 'cdd']>0, 'cool_random'],
                marker='o', linewidth = 0, color=colours.blue)

ax[3].set_ylabel('Cooling loads, Geneva [$kWh/m^2$]')
ax[4].set_xlabel('HDD')
ax[5].set_xlabel('CDD')

# ax[0].set_ylim([0, 300])

ax[0].set_xticks(np.arange(1,13,1))
ax[0].set_xticklabels([x[:3] for x in df_ddn_monthly.index.month_name()], rotation=90)
ax[3].set_xticks(np.arange(1,13,1))
ax[3].set_xticklabels([x[:3] for x in df_ddn_monthly.index.month_name()], rotation=90)

plt.show()

<h3> Questions</h3>
<ul>
    <li> Is the profile of loads broadly correct? </li>
    <li> Why do some of the plots look linear? </li>    
</ul>

<h3> Random coefficients, Geneva</h3>

Let's plot again with data from Geneva. Then, we can compare the model outputs with results from actual simulations, to show why this approach is rubbish.

In [ ]:
cdd_temp_list = list()
hdd_temp_list = list()
index_temp_list = list()
cool_temp_list = list()
heat_temp_list = list()

for y in df_gen.index.year.unique():
    for m in df_gen.index.month.unique():
        
        time_filter = np.logical_and(df_gen.index.year==y, df_gen.index.month==m)
        
        cdd_temp, hdd_temp = dd(df_gen.loc[time_filter,'tdb'],
                         bp, freq='H')
        
        if cdd_temp<=tol:
            cdd_temp = 0
        if hdd_temp<=tol:
            hdd_temp = 0
    
        index_temp = pd.to_datetime(f'{y}-{m}-01 00:00')
        
        cdd_temp_list.append(cdd_temp)
        hdd_temp_list.append(hdd_temp)
        index_temp_list.append(index_temp)
    
        # Calculate the monthly energy consumption with random coefficients.
        cool_temp_list.append(loads_linear([cdd_temp, hdd_temp], [random_coeffs[0], random_coeffs[1], 0]))
        heat_temp_list.append(loads_linear([cdd_temp, hdd_temp], [random_coeffs[0], 0, random_coeffs[2]]))        
     
df_gen_monthly = pd.DataFrame(dict(cdd=cdd_temp_list, hdd=hdd_temp_list, heat_random=heat_temp_list, cool_random=cool_temp_list),index=index_temp_list)

# Sort dataframe by year.
df_gen_monthly.sort_index(inplace=True)

In [ ]:
# Load simulation results.
loads_hourly = pickle.load(open('../Switzerland_Geneva/total_hourly_loads.p', 'rb'))

# The typical years in this data are tagged with the year 1900 but the weather data uses year 2223,
# because the datasets are based on different versions of my code.
# Anyhow, we drop the typical data for the fitting step.
loads_hourly.drop(loads_hourly[loads_hourly.index.year==1900].index, inplace=True)
loads_hourly.sort_index(inplace=True)

loads_monthly = loads_hourly.groupby([loads_hourly.index.year, loads_hourly.index.month]).sum()
loads_monthly.index = df_gen_monthly[df_gen_monthly.index.year!=2223].index


In [ ]:
# Quick inspection of uploaded data.
print(loads_hourly.describe())
loads_hourly.hist()

In [ ]:
# Does monthly data make sense?
print(loads_monthly.describe())
loads_monthly.hist()

In [ ]:
# Drop typical years from df_gen_monthly dataframe
df_gen_monthly.drop(df_gen_monthly[df_gen_monthly.index.year==2223].index, inplace=True)

# Add simulated loads (loads_monthly), converting them from J/m2 to kWh/m2.
df_gen_monthly = pd.concat([df_gen_monthly,pd.DataFrame(dict(heat_sim=loads_monthly['Heating']/(3.6e6),
         cool_sim=loads_monthly['Cooling']/(3.6e6)))], sort=True, axis=1)

# # Calculate error between simulated loads (ground truth) and loads calculated with random inputs.
df_gen_monthly.loc[:,'heat_error_random'] = df_gen_monthly.loc[:,'heat_sim']-df_gen_monthly.loc[:,'heat_random']
df_gen_monthly.loc[:,'cool_error_random'] = df_gen_monthly.loc[:,'cool_sim']-df_gen_monthly.loc[:,'cool_random']

In [ ]:
# Calculate error histograms for plotting later

hist_err = dict(heating=None, cooling=None)

bins_random = np.arange(0, 1000, 50)
hist_err['heat_random'], _ = epdf(df_gen_monthly['heat_error_random'].values, bins=bins_random)
hist_err['cool_random'],_ = epdf(df_gen_monthly['cool_error_random'].values, bins=bins_random)

hist_err = pd.DataFrame(hist_err)

In [ ]:
fix, axes = plt.subplots(nrows=2, ncols=3, sharex=False, sharey=False, figsize=(16, 12)) 
ax = axes.flatten()
fig.tight_layout(pad=3, w_pad=2, h_pad=5)

ax[0].set_ylabel('Heating load (sim), Geneva [$kWh/m^2$]')
e1 = ax[0].bar(df_gen_monthly.index.month, df_gen_monthly.loc[:, 'heat_sim'], color=colours.blackest)

h1 = ax[1].plot(df_gen_monthly.loc[:, 'heat_random'],
                df_gen_monthly.loc[:, 'heat_sim'],
                marker='o', linewidth = 0, color=colours.red)
c1 = ax[2].bar(bins_random[1:], hist_err.loc[:, 'heat_random'], width=25, color=colours.red)

ax[3].set_ylabel('Cooling load(sim), Geneva [$kWh/m^2$]')
e1 = ax[3].bar(df_gen_monthly.index.month, df_gen_monthly.loc[:, 'cool_sim'], color=colours.blackest)

h1 = ax[4].plot(df_gen_monthly.loc[:, 'cool_random'],
                df_gen_monthly.loc[:, 'cool_sim'],
                marker='o', linewidth = 0, color=colours.blue)
c1 = ax[5].bar(bins_random[1:], hist_err.loc[:, 'cool_random'], width=25, color=colours.blue)

ax[0].set_xticks(np.arange(1,13,1))
ax[0].set_xticklabels([x[:3] for x in df_gen_monthly.index.month_name()], rotation=90)
ax[3].set_xticks(np.arange(1,13,1))
ax[3].set_xticklabels([x[:3] for x in df_gen_monthly.index.month_name()], rotation=90)

plt.show()

<h2>Sensible coefficients, fixed inputs</h2>

Repeat the above exercise with more meaningful coefficients for the various weather parameters. We will use optimisation to fit the equation to the following simulation data. What kind of relationship do you expect to see between the degree days and loads?

In [ ]:
# Normalise all input and output values first.
# Ask me about why this step is important.
   
xymeans = df_gen_monthly.mean()
xystdevs = df_gen_monthly.std()

df_scaled_gen = (df_gen_monthly - xymeans) / xystdevs

# This should give us zero means and unit variance/stdev.
print(df_scaled_gen.describe())

popt = dict(heat=None, cool=None)
pcov = dict(heat=None, cool=None)

# Call a curve_fit function.
popt['heat'], pcov['heat'] = curve_fit(loads_linear, 
                       [df_scaled_gen.loc[:, 'cdd'].values, df_scaled_gen.loc[:, 'hdd'].values], 
                       df_scaled_gen.loc[:,'heat_sim'].values,
                       p0=np.zeros(3))
popt['cool'], pcov['cool'] = curve_fit(loads_linear, 
                       [df_scaled_gen.loc[:, 'cdd'].values, df_scaled_gen.loc[:, 'hdd'].values], 
                       df_scaled_gen.loc[:,'cool_sim'].values,
                       p0=np.zeros(3).tolist())

In [ ]:
# Calculate the monthly energy consumption with 'optimised' coefficients.

df_scaled_gen.loc[:,'heat_fit'] = loads_linear(
    [df_scaled_gen.loc[:,'cdd'].values, df_scaled_gen.loc[:,'hdd']], popt['heat'].tolist())
df_scaled_gen.loc[:,'cool_fit'] = loads_linear(
    [df_scaled_gen.loc[:,'cdd'].values, df_scaled_gen.loc[:,'hdd']], popt['cool'].tolist())

# Use the mean and stdev of the original y data to rescale the calculated loads.
xymeans['heat_fit'] = xymeans['heat_sim']
xystdevs['heat_fit'] = xystdevs['heat_sim']
xymeans['cool_fit'] = xymeans['cool_sim']
xystdevs['cool_fit'] = xystdevs['cool_sim']

df_gen_monthly.loc[:,'heat_fit'] = (df_scaled_gen.loc[:,'heat_fit'] * xystdevs['heat_fit']) + xymeans['heat_fit']
df_gen_monthly.loc[:,'cool_fit'] = (df_scaled_gen.loc[:,'cool_fit'] * xystdevs['cool_fit']) + xymeans['cool_fit']

df_gen_monthly.loc[df_gen_monthly.loc[:,'heat_fit']<0,'heat_fit'] = 0
df_gen_monthly.loc[df_gen_monthly.loc[:,'cool_fit']<0,'cool_fit'] = 0

df_gen_monthly.loc[:,'heat_error_fit'] = df_gen_monthly.loc[:,'heat_sim'] - df_gen_monthly.loc[:,'heat_fit']
df_gen_monthly.loc[:,'cool_error_fit'] = df_gen_monthly.loc[:,'cool_sim'] - df_gen_monthly.loc[:,'cool_fit']


In [ ]:
# Calculate error histograms for plotting later

hist_err = pd.DataFrame()

bins_fit = np.arange(-10000,10500,500)

hist_err.loc[:,'heat_fit'],_ = rel_hist(np.clip(df_gen_monthly['heat_error_fit'], bins_fit[0], bins_fit[-1]),bins=bins_fit)
hist_err.loc[:,'cool_fit'],_ = rel_hist(np.clip(df_gen_monthly['cool_error_fit'], bins_fit[0], bins_fit[-1]),bins=bins_fit)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16, 12)) 
ax = axes.flatten()
fig.tight_layout(pad=5, w_pad=5, h_pad=5)

fig.suptitle('Heating and cooling loads, Geneva')

width=0.5

ax[0].bar(df_gen_monthly.index.month.unique(),
          df_gen_monthly.loc[:,'heat_fit'].groupby(by=df_gen_monthly.index.month).median(),
          alpha=1, zorder=101, color=colours.grey, width=width)
ax[0].bar(df_gen_monthly.index.month.unique()+width,
          df_gen_monthly.loc[:,'heat_sim'].groupby(by=df_gen_monthly.index.month).median(),
          alpha=1, zorder=100, color=colours.red, width=width)
ax[1].plot(df_gen_monthly.loc[:,'heat_sim'], df_gen_monthly.loc[:,'heat_fit'],
           linewidth=0, marker='o', color=colours.red)
ax[2].bar(bins_fit[1:], hist_err.loc[:, 'heat_fit'],
         width=500, color=colours.grey)

ax[3].bar(df_gen_monthly.index.month.unique(),
          df_gen_monthly.loc[:,'cool_fit'].groupby(by=df_gen_monthly.index.month).median(),
          alpha=1, zorder=101, color=colours.grey, width=width)
ax[3].bar(df_gen_monthly.index.month.unique()+width,
          df_gen_monthly.loc[:,'cool_sim'].groupby(by=df_gen_monthly.index.month).median(),
          alpha=1, zorder=100, color=colours.blue, width=width)
ax[4].plot(df_gen_monthly.loc[:,'cool_sim'], df_gen_monthly.loc[:,'cool_fit'],
           linewidth=0, marker='o', color=colours.blue)
ax[5].bar(bins_fit[1:], hist_err.loc[:, 'cool_fit'],
         width=500, color=colours.grey)

ax[0].set_ylabel('Heating energy [$kWh/m^2$]')
ax[0].set_xticks(np.arange(1,13))
ax[0].set_xticklabels([x[:3] for x in df_gen_monthly.index.month_name()], rotation=90)
ax[1].set_ylabel('Heating energy (fit) [$kWh/m^2$]')
ax[1].set_xlabel('Heating energy (sim) [$kWh/m^2$]')
ax[2].set_xlabel('Error (sim-fit)[$kWh/m^2$]')
ax[1].set_xlim([0,60000])
ax[1].set_ylim([0,60000])
ax[2].set_xticks(np.append(bins_fit[0:-1:10], bins_fit[-1]))
ax[2].set_xticklabels(np.append(bins_fit[0:-1:10], bins_fit[-1]), rotation = 45, ha="right")

ax[3].set_ylabel('Cooling energy [$kWh/m^2$]')
ax[3].set_xticks(np.arange(1,13))
ax[3].set_xticklabels([x[:3] for x in df_gen_monthly.index.month_name()], rotation=90)
ax[4].set_ylabel('Cooling energy (fit) [$kWh/m^2$]')
ax[4].set_xlabel('Cooling energy (sim) [$kWh/m^2$]')
ax[5].set_xlabel('Error (sim-fit)[$kWh/m^2$]')
ax[4].set_xlim([0,40000])
ax[4].set_ylim([0,40000])
ax[5].set_xticks(np.append(bins_fit[0:-1:10], bins_fit[-1]))
ax[5].set_xticklabels(np.append(bins_fit[0:-1:10], bins_fit[-1]), rotation = 45, ha="right")

plt.show()

## Fixed coefficients, random inputs

Now let's try the same exercise with fixed coefficients and random inputs.


In [ ]:
# Load the synthetic future time series.
path_to_syn = '../Switzerland_Geneva/synthetic/syn_2051_2060_red.p'
synlist = pickle.load(open(path_to_syn, 'rb'))

df_syn = pd.concat(synlist)

In [ ]:
# Number of years in dataset.
future_years = list()

for df in synlist:
    future_years.extend((df['year'].unique().tolist()))

n_years = (np.unique(future_years)).size
n_hours = 8760  # Fixed for this dataset.

In [ ]:
temp = df_syn.loc[:,'tdb'].groupby(
    [df_syn.index.year, df_syn.index.month]).apply(lambda x: dd(x, bp, freq='H'))

df_syn_monthly = pd.DataFrame(temp.values.tolist(), columns=['cdd','hdd'],
                              index=pd.date_range(start='2051-01-31', end='2060-12-31', freq='1M'))

df_syn_monthly.loc[:, 'tdb'] = df_syn.loc[:, 'tdb'].groupby(
    [df_syn.index.year, df_syn.index.month]).mean().values

In [ ]:
# Now let's fix the coefficients to some 'known' values (from the linear fit above)
# and plot the outputs using 'random' inputs.

# xymeans_syn = df_syn_monthly.mean()
# xystdevs_syn = df_syn_monthly.std()

df_syn_scaled = (df_syn_monthly - df_syn_monthly.mean()) / df_syn_monthly.std()
   
df_syn_scaled['heat_fit'] = loads_linear(
    [df_syn_scaled.loc[:,'cdd'], df_syn_scaled.loc[:,'hdd']], popt['heat'].tolist())
df_syn_scaled['cool_fit'] = loads_linear(
    [df_syn_scaled.loc[:,'cdd'].values, df_syn_scaled.loc[:,'hdd']], popt['cool'].tolist())

# # xymeans_syn['energy_fit'] = xymeans['energy_fit']
# # xystdevs_syn['energy_fit'] = xystdevs['energy_fit']

df_syn_monthly['heat_fit'] = (df_syn_scaled['heat_fit'] * xystdevs['heat_fit']) + xymeans['heat_fit']
df_syn_monthly['cool_fit'] = (df_syn_scaled['cool_fit'] * xystdevs['cool_fit']) + xymeans['cool_fit']

In [ ]:
# I'm leaving this plot incomplete for you to practice.

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 12)) 
ax = axes.flatten()
fig.tight_layout(pad=3, w_pad=2, h_pad=5)

ax[0].set_title('Monthly energy usage, Geneva [$kWh/m^2$]')
ax[1].set_title('Future temperature, Geneva [$\circ$C]')

ax[0].plot(df_syn_monthly.loc[:,'heat_fit'], alpha=1, zorder=103, color=colours.red)
ax[0].plot(df_syn_monthly.loc[:,'cool_fit'], alpha=1, zorder=104, color=colours.blue)

ax[1].plot(df_syn_monthly.loc[:,'tdb'], alpha=1, zorder=103, color=colours.blackest)

ax[0].set_ylabel('Energy / [kWh]')
ax[1].set_ylabel('Temperature / [$\circ$C]')

plt.show()

## Uncertainty analysis with EPlus

### Steps/Notes

1. Consider weather inputs from Indra.
2. Run samples using EPlus - analyse results.
    1. Either modify a base EPlus file using Eppy/scripting or in jePlus.
    2. Or analyse the same EPlus file using many weather files.

<h2> Bibliography </h2>

<ul>
<li>Rastogi, Parag. 2016. “On the Sensitivity of Buildings to Climate: The Interaction of Weather and Building Envelopes in Determining Future Building Energy Consumption.” PhD, Lausanne, Switzerland: Ecole polytechnique fédérale de Lausanne. EPFL Infoscience. https://infoscience.epfl.ch/record/220971?ln=en.
<li>Rastogi, Parag, and Marilyne Andersen. 2015. “Embedding Stochasticity in Building Simulation Through Synthetic Weather Files.” In Proceedings of BS 2015. Hyderabad, India. http://infoscience.epfl.ch/record/208743.
<li>———. 2016. “Incorporating Climate Change Predictions in the Analysis of Weather-Based Uncertainty.” In Proceedings of SimBuild 2016. Salt Lake City, UT, USA. http://infoscience.epfl.ch/record/208743.
<li>Rastogi, Parag, Mohammad Emtiyaz Khan, and Marilyne Andersen. 2017. “Gaussian-Process-Based Emulators for Building Performance Simulation.” In Proceedings of BS 2017. San Francisco, CA, USA: IBPSA.
<li>Iaccarino, Gianluca. 2008. “Quantification of Uncertainty in Flow Simulations Using Probabilistic Methods.” presented at the VKI Lecture Series, Stanford University, September. http://web.stanford.edu/group/uq/uq_youq.html.
<li>Macdonald, Iain. 2002. “Quantifying the Effects of Uncertainty in Building Simulation.” Doctoral, University of Strathclyde. https://www.strath.ac.uk/media/departments/mechanicalengineering/esru/research/phdmphilprojects/macdonald_thesis.pdf.

</ul>